# Imports


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
from time import time
from sp_sims.simulators.stochasticprocesses import BDStates
from samprecon.environments.OneEpisodeEnvironments import MarkovianUniformEnvironment
from samprecon.samplers.agents import SimpleAgent 
from samprecon.reconstructors.NNReconstructors import RNNReconstructor
from tqdm.notebook import tqdm

plt.style.use('rose-pine-dawn')
rnd.seed(int(time()))
%load_ext autoreload
%autoreload 2

In [2]:
# Generate Environments on which to learn 
high_res_delta = 1e-1 # For generating the dataset and later sample
baseline_rates = {"lam": 1/10,"mu":4/10}
epochs = 100
lenth_of_episode = 12
step_path_length = 1
sampling_budget = 4
used_path_length = 64 # So that we can let the process reach stationarity and take samples from stationary distribution
num_states = 4
avg_span = np.mean(1/np.array(
            list(baseline_rates.values())
    ))
max_decimation = (avg_span/high_res_delta)*4 # Max decimation factor #CHECK: Maybe not divide by 2
current_decimation_factor = int(# We can start somewhere in between 
    avg_span//high_res_delta)
print(f"Decimation factor is {current_decimation_factor}")
# Set random seed with time for randomnessj

Decimation factor is 62


In [3]:
# Initialize context first
stateGen = BDStates(baseline_rates, high_res_delta, num_states)
sampling_arbiter = SimpleAgent(num_states,max_decimation)
reconstructor = RNNReconstructor(amnt_states = num_states)

env = MarkovianUniformEnvironment(
    state_generator = stateGen,
    sampling_arbiter = sampling_arbiter,
    reconstructor = reconstructor, 
    starting_decrate = current_decimation_factor,
)

In [5]:
ebar = tqdm(range(epochs), desc = "Epochs", position=0)
e_losses = []
for epoch in range(epochs):
    # We generate a single step from the generator process
    leave = epoch == len(epochs) - 1
    sbar = tqdm(range(lenth_of_episode), desc = "Steps", leave=leave, position=1)
    for step in range(lenth_of_episode):

        loss = env.step() 
        sbar.set_description(f"Loss: {loss}")

        sbar.update(1)
    e_losses = np.append(e_losses,loss)
    ebar.set_description(f"Epoch Mean Loss: {np.mean(e_losses[:-10])}")
    # We get reward based on how close we got to maximum information 

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

NameError: name 'outter_range' is not defined